# 문제1

### **문제 1) Tokenizer 생성하기**

**1-1. `preprocessing()`**

텍스트 전처리를 하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장을 토큰화한 결과로, nested list 형태입니다. ex) [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]
- 조건 1: 입력된 문장에 대해서 소문자로의 변환과 특수문자 제거를 수행합니다.
- 조건 2: 토큰화는 white space 단위로 수행합니다.
    
    

**1-2. `fit()`**

어휘 사전을 구축하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: 위에서 만든 `preprocessing` 함수를 이용하여 각 문장에 대해 토큰화를 수행합니다.
- 조건 2: 각각의 토큰을 정수 인덱싱 하기 위한 어휘 사전(`self.word_dict`)을 생성합니다.
    - 주어진 코드에 있는 `self.word_dict`를 활용합니다.
    

**1-3. `transform()`**

어휘 사전을 활용하여 입력 문장을 정수 인덱싱하는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장의 정수 인덱싱으로, nested list 형태입니다. ex) [[1, 2, 3, 4], [1, 5, 6]]
- 조건 1: 어휘 사전(`self.word_dict`)에 없는 단어는 'oov'의 index로 변환합니다.

In [44]:
import string

class Tokenizer():
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False
  
  def preprocessing(self, sequences):
    result = []
    for s in sequences:
        new_s = s.lower().translate(str.maketrans('', '', string.punctuation))
        arr = list(new_s.split(' '))
        result.append(arr)
    return result
  
  def fit(self, sequences):
    self.fit_checker = False
    arr = self.preprocessing(sequences)
    arr2 = list(set(sum(arr, [])))
    token_index = 1
    for t in arr2:
        self.word_dict.update({t: token_index})
        token_index+=1  
    self.fit_checker = True
    print(f"fitted word dict: {self.word_dict}")
  
  def transform(self, sequences):
    result = []
    tokens = self.preprocessing(sequences)
    if self.fit_checker:
        for token in tokens:
            arr = []
            for t in token:
                trans_t = self.word_dict.get(t)
                if trans_t != None:
                    arr.append(trans_t)
                else:
                    arr.append(self.word_dict['oov'])
            result.append(arr)
        
        
  
        return result
    else:
      raise Exception("Tokenizer instance is not fitted yet.")
      
  def fit_transform(self, sequences):
    self.fit(sequences)
    result = self.transform(sequences)
    return result




In [46]:
example_set = ['I go to school.', 'I LIKE pizza!']

token = Tokenizer()

print("문제 1-1")
arr = token.preprocessing(example_set)
print(f"전처리 결과값: {arr}")

print("문제 1-2")
token.fit(example_set)

result = token.transform(example_set)
print("문제 1-3")
print(f"결과값: {result}")

문제 1-1
전처리 결과값: [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]
문제 1-2
fitted word dict: {'oov': 0, 'like': 1, 'go': 2, 'i': 3, 'to': 4, 'pizza': 5, 'school': 6}
문제 1-3
결과값: [[3, 2, 4, 6], [3, 1, 5]]


# 문제2

### **문제 2) TfidfVectorizer 생성하기**

**2-1. `fit()`**

입력 문장들을 이용해 IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: IDF 행렬은 list 형태입니다.
    - ex) [토큰1에 대한 IDF 값, 토큰2에 대한 IDF 값, .... ]
- 조건 2: IDF 값은 아래 식을 이용해 구합니다.
    
    $$
    idf(d,t)=log_e(\frac{n}{1+df(d,t)})
    $$
    
    - $df(d,t)$ : 단어 t가 포함된 문장 d의 개수
    - $n$ : 입력된 전체 문장 개수
- 조건 3: 입력된 문장의 토큰화에는 문제 1에서 만든 Tokenizer를 사용합니다.
    
    

**2-2. `transform()`**

입력 문장들을 이용해 TF-IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output : nested list 형태입니다.
    
    ex) [[tf-idf(1, 1), tf-idf(1, 2), tf-idf(1, 3)], [tf-idf(2, 1), tf-idf(2, 2), tf-idf(2, 3)]]
    
    |  | 토큰1 | 토큰2 | 토큰3 |
    | --- | --- | --- | --- |
    | 문장1 | tf-idf(1,1) | tf-idf(1,2) | tf-idf(1,3) |
    | 문장2 | tf-idf(2,1) | tf-idf(2,2) | tf-idf(2,3) |
- 조건1 : 입력 문장을 이용해 TF 행렬을 만드세요.
    - $tf(d, t)$ : 문장 d에 단어 t가 나타난 횟수
- 조건2 : 문제 2-1( `fit()`)에서 만든 IDF 행렬과 아래 식을 이용해 TF-IDF 행렬을 만드세요
    
    $$
    tf-idf(d,t) = tf(d,t) \times idf(d,t)
    $$

In [47]:
class TfidfVectorizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
    self.fit_checker = False
  
  def fit(self, sequences):
    tokenized = self.tokenizer.fit_transform(sequences)
    '''
    문제 2-1.
    '''
    self.fit_checker = True
    

  def transform(self, sequences):
    if self.fit_checker:
      tokenized = self.tokenizer.transform(sequences)
      '''
      문제 2-2.
      '''
      return self.tfidf_matrix
    else:
      raise Exception("TfidfVectorizer instance is not fitted yet.")

  
  def fit_transform(self, sequences):
    self.fit(sequences)
    return self.transform(sequences)
  def get_idf(word, seq):
        word_dict = self.tokenizer.word_dict 
        target_token = word_dict[word]
        value = 0
        for s in seq:
            if target_token in s:
                value += 1;
            else:
                continue
            
        result = len(seq) / (1 + value)
        return pass
    

In [42]:
import os

cwd = os.getcwd()

with open(f'{cwd}/random_sentences.txt') as f:
    test_set = f.read().splitlines() 
    
print(test_set)



['She saw no irony asking me to change but wanting me to accept her for who she is.', 'He was all business when he wore his clown suit.', 'He stepped gingerly onto the bridge knowing that enchantment awaited on the other side.', "It doesn't sound like that will ever be on my travel list.", 'The near-death experience brought new ideas to light.', 'She found it strange that people use their cellphones to actually talk to one another.', 'Poison ivy grew through the fence they said was impenetrable.', "She wrote him a long letter, but he didn't read it.", 'Karen believed all traffic laws should be obeyed by all except herself.', 'His thought process was on so many levels that he gave himself a phobia of heights.', 'As the asteroid hurtled toward earth, Becky was upset her dentist appointment had been canceled.', 'The urgent care center was flooded with patients after the news of a new deadly virus was made public.', 'When motorists sped in and out of traffic, all she could think of was tho